## Import modules

In [132]:
import pandas as pd
import numpy as np

## Load data

In [133]:
"""
Need to provide location of the file to load.
Structure of the file:
    - .csv format
    - column label in the first raw
    - possible multiple columns
    
Loaded data:
    - firstly as pandas dataframe
    - to make the algorithm faster - transformed to numpy array
"""

def load_data(file_location, file_name):
    data = pd.read_csv(file_location + file_name)
    return data

## Function 1: Scan for possible change point

In [139]:
# --- Find possible change point ---

def no_cps(start, step, max_length, changepoints):
    """
    --- Find possible change point ---

    Args:
                    start: int. Start of the iteration
                    step: int. Iteration step
                    cps2: int. Time position of the last changepoint.

    Task: 
                    Scan for possible change point.

    """
    
    for i in range(start, len(data) - 1, step):     
    
        # Statistics 1
        data_statistics = data[changepoints:i]
        mu0_1 = np.mean(data_statistics)
        n_1 = len(data_statistics)
        s0_1 = [np.sum((data_statistics - mu0_1) ** 2)] * (n_1-1)
        s1_1 = np.asarray([np.sum((data_statistics[0:i] - np.mean(data_statistics[0:i])) ** 2) for i in range(1, n_1)])
        s2_1 = np.asarray([np.sum((data_statistics[i:] - np.mean(data_statistics[i:])) ** 2) for i in range(1, n_1)])
        R_1  = s0_1 - s1_1 - s2_1
        G_1  = np.max(R_1)
        taustar_1 = int(np.max(np.where(R_1 == G_1))) + step

        # Statistics 2
        data_statistics = data[changepoints:i + step]
        mu0_2 = np.mean(data_statistics)
        n_2 = len(data_statistics)
        s0_2 = [np.sum((data_statistics - mu0_2) ** 2)] * (n_2 - 1)
        s1_2 = np.asarray([np.sum((data_statistics[0:i] - np.mean(data_statistics[0:i])) ** 2) for i in range(1, n_2)])
        s2_2 = np.asarray([np.sum((data_statistics[i:] - np.mean(data_statistics[i:])) ** 2) for i in range(1, n_2)])
        R_2  = s0_2 - s1_2 - s2_2
        G_2  = np.max(R_2)
        taustar_2 = int(np.max(np.where(R_2 == G_2))) + step
        
        # Propagate end position of the searching period
        end = i
    
        # Case 1: no change point G_2 > G_2
        if G_2 > G_1:
            print('Stationarity', 'dG:', round(G_2 - G_1, 4), 
                  ', start:', start,  
                  ', last cps: ', changepoints, 
                  ', end:', end
                 )
            
            # Case 1, additional condition 1: 
            # if the searching period is larger than break_condition 
            # - force change point
            if end - start > max_length or end - changepoints > max_length:
                cps_terminate = end # cps3: forced changepoint as end position
                return G_1, G_2, cps_terminate, end
            
            # Case 1, additional condition 3: 
            # if the searching comes to the end of time series length, 
            # - terminate the function
            if end == len(data):
                cps_terminate = 0
                return G_1, G_2, cps_terminate, end
            
            pass

        # Case 2: possible change point detected G_2 < G_1
        elif G_2 < G_1:
            print('Possible change point', 'dG:', round(G_2 - G_1, 4), 
                  ', start: ', start,  
                  ', last cps: ', changepoints, 
                  ', end: ', end
                 )
            cps_terminate = 0
            return G_1, G_2, cps_terminate, end

## Function 2: Check whether detected change point is a true change point

In [140]:
# --- Check whether the possible change point is a true changepoint ---
# --- Function input varialbes ---

def cps(start, G_1, step, max_length, changepoints):
    
    """
    --- Find possible change point ---

    Args:
                    start: int. Start of the iteration
                    G_1: int. Previously calculated G_2 at possible change point assigned as G_1
                    step: int. Iteration step
                    cps2: int. Time position of the last changepoint.

    Task: 
                    Check whether previously detected change point is a true change point

    """    
    cps_pos = []
    for i in range(start, len(data) - 1, step):

        # Statistics 2
        data_statistics = data[changepoints:i+step]
        mu0_2 = np.mean(data_statistics)
        n_2 = len(data_statistics)
        s0_2 = [np.sum((data_statistics-mu0_2) ** 2)] * (n_2 - 1)
        s1_2 = np.asarray([np.sum((data_statistics[0:i] - np.mean(data_statistics[0:i])) ** 2) for i in range(1, n_2)])
        s2_2 = np.asarray([np.sum((data_statistics[i:] - np.mean(data_statistics[i:])) ** 2) for i in range(1, n_2)])
        R_2  = s0_2 - s1_2 - s2_2
        G_2  = np.max(R_2)
        taustar_2 = int(np.max(np.where(R_2 == G_2))) + step
        
        # Propagate end position of the searching period and create array for change point calculation
        end = i
        cps_pos.append(end)
        
        # --- Case 1: Possible change point can be a change point - scanning, G_2 < G_1
        if G_2 < G_1:
            print('Non-stationarity', 'dG:', round(G_2 - G_1, 4), 
                  ', start:', start, 
                  ', last cps: ', changepoints, 
                  ', end:', end
                 )
            
            # --- Case 1, additional condition 1: 
            # if the checking period (end:start) is larger than break_condition
            # - force change point            
            if end - start > max_length or end - changepoints > max_length:
                cps_terminate = end
                cps_pos = np.zeros(2)
                print('cps3:', cps_terminate
                     )
                return G_1, G_2, cps_terminate, end, cps_pos
            
            continue
        
        # --- Case 2: True change point detected - after searching, checking G_2 > G_1
        elif G_2 > G_1:
            print('Change point detected', 'dG:', round(G_2 - G_1, 4), 
                  'start', start,  
                  'cps2_start: ', changepoints, 
                  'end', end
                 )
            cps_terminate = 0
            return G_1, G_2, cps_terminate, end, cps_pos

## Function 3: Scan the whole dataset for change points and approximate steady stataes

In [141]:
def find_cps(filepath, filename, step, min_length, max_length):
    
    """
    --- Scan the whole dataset and find all change points  ---

    Args:
                    filepath: location of the folder with input data
                    filename: name of the file with data
                    step: step between scanned points (for example step = 1 takes every points, step = 2, takes every two points)
                    min_length: minimum length of steady states
                    max_length: maximum length of steady states

    Task: 
                    Detect all change points in the dataset, and approximate steady states as mean constant
    
    """
    
    # Initiate empty dataframes for data export
    data_smooth = pd.DataFrame([])
    data_changepoints = pd.DataFrame([])
    
    # Import data
    data_input = load_data(filepath, filename)
    
    # Labels in the data and convert them into numpy array (it is faster for big data)
    labels = data_input.columns
    data_input = data_input.to_numpy()
    
    # Analyse all imported data
    for k, name in enumerate(labels):
        print('Currently processed: ', name)

        """
        G_1: statistics 1
        G_2: statistics 2
        start: starting point
        changepoints: list of changepoints
        length: length of potential steady state
        data_smoothed: smoothed data at the end of the loop
        """
        
        G_1 = 0.1
        G_2 = 0.2
        start = 10
        changepoints = np.zeros(1, dtype ='int')
        data_smooth_2 = []
        
        data = data_input[:, k]
        
        while start < len(data):

            # Parameters
            cps_pos = []
            cps_terminate = 0
            length = 0

            # Case 1: No change point
            if G_2 > G_1:
                print('Start change point search')
                G_1, G_2, cps_terminate, end = no_cps(start, step, max_length, changepoints[-1])
                start = end + step

            # Case 2: Checking the potential change point
            elif G_2 < G_1:
                print('Start change point checking')
                G_1, G_2, cps_terminate, end, cps_pos = cps(start, G_2, step, max_length, changepoints[-1])
                length = max(cps_pos) - min(cps_pos)
                start = end + step

            # Propagating change point from termination
            if cps_terminate > 0:
                changepoints = np.append(changepoints, cps_terminate)
                start = cps_terminate + step

            # Checking if the detected change point satisfy the minimum length of steady states
            if length > min_length:
                changepoints = np.append(changepoints, np.max(cps_pos))
                start = changepoints[-1] + step

        # Removing duplicates    
        changepoints = np.unique(changepoints)

        # Creating a numpy array with approximated constant steady states up to the last change point
        for i in range(0, len(changepoints) - 1):
            mean = np.mean(data[changepoints[i]:changepoints[i + 1]])
            data_cut = (np.ones(changepoints[i + 1] - changepoints[i])) * mean
            data_smooth_2 = np.append(data_smooth_2, data_cut)

        # Padding the end of created approximations to be the same length as input data
        if len(data_smooth_2) < len(data):
            filler = np.ones(len(data) - len(data_smooth_2)) * data_smooth_2[-1]
            data_smooth_2 = np.append(data_smooth_2, filler)

        # Convert to dataframe and export as .csv file
        data_smooth_2 = pd.DataFrame(data = data_smooth_2, columns = [name])
        changepoints = pd.DataFrame(data = changepoints, columns = [name])
        data_smooth = pd.concat([data_smooth, data_smooth_2], axis = 1)
        data_changepoints = pd.concat([data_changepoints, changepoints], axis = 1)
        data_smooth.to_csv(file_location + 'output_smooth.csv')
        data_changepoints.to_csv(file_location + 'output_changepoints.csv')

## Example how to use this method

In [142]:
file_location = 'C:/Users/kamil/Dropbox (The University of Manchester)/KTP/GitHub/'
file_name = 'test_data.csv'

find_cps(file_location, file_name, 100, 120, 480)

Currently processed:  test1
Start change point search
Stationarity dG: 0.0251 , start: 10 , last cps:  0 , end: 10
Stationarity dG: 0.021 , start: 10 , last cps:  0 , end: 110
Stationarity dG: 0.0143 , start: 10 , last cps:  0 , end: 210
Stationarity dG: 0.007 , start: 10 , last cps:  0 , end: 310
Stationarity dG: 0.0553 , start: 10 , last cps:  0 , end: 410
Possible change point dG: -0.015 , start:  10 , last cps:  0 , end:  510
Start change point checking
Non-stationarity dG: -0.0035 , start: 610 , last cps:  0 , end: 610
cps3: 610
Start change point checking
Non-stationarity dG: -0.0758 , start: 710 , last cps:  610 , end: 710
Non-stationarity dG: -0.0636 , start: 710 , last cps:  610 , end: 810
Non-stationarity dG: -0.0689 , start: 710 , last cps:  610 , end: 910
Change point detected dG: 1.9429 start 710 cps2_start:  610 end 1010
Start change point search
Possible change point dG: -0.0208 , start:  1110 , last cps:  1010 , end:  1110
Start change point checking
Non-stationarity dG